In [1]:
!cd /Users/lc/Developer/kuzu &&  make clean-python-api && make release NUM_THREADS=20 

rm -rf tools/python_api/build
cd external && \
	mkdir -p build && \
	cd build && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../ && \
	cmake --build . --config Release -- -j 20
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/external/build
[100%] Built target apache_arrow
mkdir -p build/release && \
	cd build/release && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../.. && \
	cmake --build . --config Release -- -j 20
-- pybind11 v2.11.0 dev1
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/build/release
[  2%] Built target utf8proc
[  2%] Built target kuzu_transaction
[  2%] Built target kuzu_common_csv_reader
[  3%] Built target kuzu_common_data_chunk
[  3%] Built target kuzu_common_task_system
[  4%] Built target kuzu_common_vector
[  4%] Built target kuzu_common_arrow
[  5%] Built target kuzu_binder_expression
[  5%] Built target kuzu_catalog
[  6%] Built target kuzu_binder
[  7%] Built tar

In [2]:
import sys
sys.path.append('/Users/lc/Developer/kuzu/tools/python_api/build')
import kuzu

In [3]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG


dataset = OGB_MAG(root='/Users/lc/Developer/ogb_pytorch', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

In [4]:
DB_PATH = '/Users/lc/Developer/kuzu_ogb_mag'
db = kuzu.Database(DB_PATH)

In [5]:
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)

In [6]:
fs = db.get_torch_geometric_remote_backend()

In [7]:
import random
import time

pyg_times = []
kuzu_times = []

n = 40
for i in range(n):
    print(i + 1, "/", n)
    ids = random.sample(range(0, len(data['paper'].x)), 1000)
    start = time.time()
    d = fs['paper'].x[ids]
    end = time.time()
    kuzu_times.append(end - start)

    start = time.time()
    d = data['paper'].x[ids]
    end = time.time()
    pyg_times.append(end - start)

1 / 40
Query time: 0.423360
Fetch time: 0.016158
Convert time: 0.002871
2 / 40
Query time: 0.383816
Fetch time: 0.013642
Convert time: 0.002725
3 / 40
Query time: 0.368383
Fetch time: 0.012736
Convert time: 0.002534
4 / 40
Query time: 0.363211
Fetch time: 0.013013
Convert time: 0.002545
5 / 40
Query time: 0.364667
Fetch time: 0.013356
Convert time: 0.002548
6 / 40
Query time: 0.364420
Fetch time: 0.014007
Convert time: 0.002737
7 / 40
Query time: 0.360448
Fetch time: 0.013478
Convert time: 0.002717
8 / 40
Query time: 0.371671
Fetch time: 0.012366
Convert time: 0.002425
9 / 40
Query time: 0.369121
Fetch time: 0.013463
Convert time: 0.002646
10 / 40
Query time: 0.370358
Fetch time: 0.014202
Convert time: 0.002519
11 / 40
Query time: 0.375803
Fetch time: 0.014336
Convert time: 0.002706
12 / 40
Query time: 0.365298
Fetch time: 0.013855
Convert time: 0.002661
13 / 40
Query time: 0.370309
Fetch time: 0.013382
Convert time: 0.002643
14 / 40
Query time: 0.361738
Fetch time: 0.014001
Convert ti

In [8]:
pyg_time = sum(pyg_times) / len(pyg_times)
kuzu_time = sum(kuzu_times) / len(kuzu_times)

print("PyG time: ", pyg_time)
print("Kuzu time: ", kuzu_time)

print(kuzu_time / pyg_time)


PyG time:  0.00045774579048156737
Kuzu time:  0.3930705785751343
858.7093245829916
